Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Extractive Summarization on CNN/DM Dataset using Transformer Version of BertSum


### Summary

This notebook demonstrates how to fine tune Transformers for extractive text summarization. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation.

BertSum refers to  [Fine-tune BERT for Extractive Summarization (https://arxiv.org/pdf/1903.10318.pdf) with [published example](https://github.com/nlpyang/BertSum/). And the Transformer version of Bertsum refers to our modification of BertSum and the source code can be accessed at (https://github.com/daden-ms/BertSum/). 

Extractive summarization are usually used in document summarization where each input document consists of mutiple sentences. The preprocessing of the input training data involves assigning label 0 or 1 to the document sentences based on the give summary. The summarization problem is also simplfied to classifying whether each document sentence should be included in the summary. 

The figure below illustrates how BERTSum can be fine tuned for extractive summarization task. Each sentence is inserted with [CLS] token at the beginning and  [SEP] at the end. Interval segment embedding and positional embedding are added upon the token embedding before input the BERT model. The [CLS] token representation is used as sentence embedding and only the [CLS] tokens are used as input for the summarization model. The summarization layer predicts whether the probability of each each sentence token should be included in the summary or not. Techniques like trigram blocking can be used to improve model accuarcy.   

<img src="https://nlpbp.blob.core.windows.net/images/BertSum.PNG">


### Before You Start

The running time shown in this notebook is on a Standard_NC24s_v3 Azure Deep Learning Virtual Machine with 4 NVIDIA Tesla V100 GPUs. 
> **Tip**: If you want to run through the notebook quickly, you can set the **`QUICK_RUN`** flag in the cell below to **`True`** to run the notebook on a small subset of the data and a smaller number of epochs. 

The table below provides some reference running time on different machine configurations.  

|QUICK_RUN|USE_PREPROCESSED_DATA|encoder|Machine Configurations|Running time|
|:---------|:---------|:---------|:----------------------|:------------|
|True|True|baseline|1 NVIDIA Tesla V100 GPUs, 16GB GPU memory| ~ 20 minutes |
|False|True|baseline|1 NVIDIA Tesla V100 GPUs, 16GB GPU memory| ~ 60 minutes |
|True|False|baseline|1 NVIDIA Tesla V100 GPUs, 16GB GPU memory| ~ 20 minutes |
|True|True|transformer|1 NVIDIA Tesla V100 GPUs, 16GB GPU memory| ~ 80 minutes |
|False|True|transformer|1 NVIDIA Tesla V100 GPUs, 16GB GPU memory| ~ 6.5hours |
|True|False|transformer|1 NVIDIA Tesla V100 GPUs, 16GB GPU memory| ~ 80 minutes |
|False|False|any| 1 NVIDIA Tesla V100 GPUs, 16GB GPU memory| > 24 hours|

In [1]:
## Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs.
QUICK_RUN = True
USE_PREPROCESSED_DATA = True

### Configuration

Before we start the notebook, we should set the environment variable to make sure you can access the GPUs on your machine

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import os
import sys
import torch
nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)
sys.path.insert(0, "./")
sys.path.insert(0, "/dadendev/nlp/examples/text_summarization/BertSum/")

from utils_nlp.common.pytorch_utils import get_device
from utils_nlp.dataset.cnndm import CNNDMBertSumProcessedData, CNNDMSummarization
from utils_nlp.eval.evaluate_summarization import get_rouge
from utils_nlp.models.transformers.extractive_summarization import (
    get_dataloader,
    get_data_iter,
    ExtractiveSummarizer,
    ExtSumProcessor,
)





[nltk_data] Downloading package punkt to /home/daden/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
I1210 21:34:47.924211 140261726336832 file_utils.py:39] PyTorch version 1.2.0 available.
I1210 21:34:47.957383 140261726336832 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
I1210 21:34:47.963832 140261726336832 modeling.py:230] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


Also, we need to install the dependencies for pyrouge.

In [7]:
# dependencies for ROUGE-1.5.5.pl
!sudo apt-get update
!sudo apt-get install expat
!sudo apt-get install libexpat-dev -y

Hit:1 http://azure.archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://azure.archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://azure.archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:4 https://packages.microsoft.com/repos/microsoft-ubuntu-xenial-prod xenial InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Ign:6 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Fetched 252 kB in 1s (392 kB/s)                              
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
expat is already the newest version (2.2.5-3ubuntu0.2).
The following packages were automatically installed and are no longer required:
  linux-azure-cloud-tools-5.0.0-1018 linux-azure-cloud-tools-5.0.0-1020
  linux-azure-headers-5.0.0-101

Run the following command in your terminal to install pre-requiste for using pyrouge.
1. sudo cpan install XML::Parser
1. sudo cpan install XML::Parser::PerlSAX
1. sudo cpan install XML::DOM

Download ROUGE-1.5.5 from https://github.com/andersjo/pyrouge/tree/master/tools/ROUGE-1.5.5.
Run the following command in your terminal.
* pyrouge_set_rouge_path $ABSOLUTE_DIRECTORY_TO_ROUGE-1.5.5.pl

### Data Preprossing

The dataset we used for this notebook is CNN/DM dataset which contains the documents and accompanying questions from the news articles of CNN and Daily mail. The highlights in each article are used as summary. The dataset consits of ~289K training examples, ~11K valiation and ~11K test dataset.  You can choose the [Option 1] below preprocess the data or [Option 2] to use the preprocessed version at [BERTSum published example](https://github.com/nlpyang/BertSum/). You don't need to manually download any of these two data sets as the code below will handle this part.  Since it takes up to 28 hours to preprocess the training data  to run on 10  Intel(R) Xeon(R) CPU E5-2690 v3 @ 2.60GHz, we suggest you continue with set as True first and experiment with data preprocessing  with QUICKRUN set as True.

##### Details of Data Preprocessing

The purpose of preprocessing is to process the input articles to the format that BertSum takes.  Functions defined specific in harvardnlp_cnndm_preprocess function are unique to CNN/DM dataset that's processed by harvardnlp. However, it provides a skeleton of how to preprocessing data into the format that BertSum takes. Assuming you have all articles and target summery each in a file, line-breaker seperated, the steps to preprocess the data are:
1. sentence tokenization
2. word tokenization
3. **label** the sentences in the article with 1 meaning the sentence is selected and 0 meaning the sentence is not selected. The options for the selection algorithms are "greedy" and "combination"
3. convert each example to  BertSum format
    - filter the sentences in the example based on the min_src_ntokens argument. If the lefted total sentence number is less than min_nsents, the example is discarded.
    - truncate the sentences in the example if the length is greater than max_src_ntokens
    - truncate the sentences in the example and the labels if the totle number of sentences is greater than max_nsents
    - [CLS] and [SEP] are inserted before and after each sentence
    - wordPiece tokenization
    - truncate the example to 512 tokens
    - convert the tokens into token indices corresponding to the BERT tokenizer's vocabulary.
    - segment ids are generated
    - [CLS] token positions are logged
    - [CLS] token labels are truncated if it's greater than 512, which is the maximum input length that can be taken by the BERT model.
    
    
Note that the original BERTSum paper use Stanford CoreNLP for data proprocessing, here we'll first how to use NLTK version, and then we also provide instruction of how to set up Stanford NLP and code examples of how to use Standford CoreNLP. 

##### [Option 1] Preprocess  data
The code in following cell will download the CNN/DM dataset listed at https://github.com/harvardnlp/sent-summary/.

In [6]:
train_dataset, test_dataset = CNNDMSummarization(top_n=top_n)

[nltk_data] Downloading package punkt to /home/daden/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
I1210 18:30:56.902205 139830036641600 file_utils.py:39] PyTorch version 1.2.0 available.
I1210 18:30:56.936184 139830036641600 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
I1210 18:30:56.943920 139830036641600 modeling.py:230] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
I1210 18:30:56.953987 139830036641600 utils.py:173] Opening tar file .data/cnndm.tar.gz.
I1210 18:30:56.965388 139830036641600 utils.py:181] .data/test.txt.src already extracted.
I1210 18:30:57.394748 139830036641600 utils.py:181] .data/test.txt.tgt.tagged already extracted.
I1210 18:30:57.430505 139830036641600 utils.py:181] .data/train.txt.src already extracted.
I1210 18:31:08.232069 139830036641600 utils.py:181] .data/train.txt.tgt.tagged already extracted.
I1210 18:31:09.120292 139830036

Preprocess the data and save the data to disk.

In [8]:
processor = ExtSumProcessor(model_name="distilbert-base-uncased")
ext_sum_train = processor.preprocess(train_dataset, train_dataset.get_target())
ext_sum_test = processor.preprocess(test_dataset, test_dataset.get_target())

I1206 16:51:11.971997 140019998881600 file_utils.py:296] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache or force_download set to True, downloading to /tmp/tmpjzna7ttv
100%|██████████| 231508/231508 [00:00<00:00, 1963987.24B/s]
I1206 16:51:12.252732 140019998881600 file_utils.py:309] copying /tmp/tmpjzna7ttv to cache at ./26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I1206 16:51:12.253886 140019998881600 file_utils.py:313] creating metadata file for ./26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I1206 16:51:12.255181 140019998881600 file_utils.py:322] removing temp file /tmp/tmpjzna7ttv
I1206 16:51:12.255890 140019998881600 tokenization_utils.py:374] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./26bc1ad6c0ac

In [7]:
data_path = "./temp_data4/"

In [10]:
!mkdir -p $data_path

In [8]:
train_files = CNNDMBertSumProcessedData.save_data(
    ext_sum_train, is_test=False, path_and_prefix=data_path, chunk_size=25
)
test_files = CNNDMBertSumProcessedData.save_data(
    ext_sum_test, is_test=True, path_and_prefix=data_path, chunk_size=None
)

[nltk_data] Downloading package punkt to /home/daden/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
I1206 21:27:24.938656 139683323610944 file_utils.py:39] PyTorch version 1.2.0 available.
I1206 21:27:24.971345 139683323610944 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
I1206 21:27:24.978915 139683323610944 modeling.py:230] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


NameError: name 'ext_sum_train' is not defined

In [12]:
train_files

['./temp_data4/_0_train',
 './temp_data4/_1_train',
 './temp_data4/_2_train',
 './temp_data4/_3_train']

In [13]:
test_files

['./temp_data4/_0_test']

In [21]:
train_iter, test_iter = CNNDMBertSumProcessedData().splits(root=data_path)

#### Inspect Data

In [15]:
import torch
bert_format_data = torch.load(train_files[0])
print(len(bert_format_data))
bert_format_data[0].keys()


25


dict_keys(['src', 'labels', 'segs', 'clss', 'src_txt', 'tgt_txt'])

The first 

In [16]:
bert_format_data[0]['labels']

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

##### [Option 2] Reuse Preprocessed  data

In [5]:
if USE_PREPROCESSED_DATA:
    data_path = "./temp_data5/"
    if not os.path.exists(data_path):
        os.mkdir(data_path)
    #CNNDMBertSumProcessedData.download(local_path=data_path)
    train_iter, test_iter = CNNDMBertSumProcessedData().splits(root=data_path)
    

### Model training
To start model training, we need to create a instance of ExtractiveSummarizer.
#### Choose the transformer model.
Currently ExtractiveSummarizer support two models:
- distilbert-base-uncase, 
- bert-base-uncase

Potentionally, roberta-based model and xlnet can be supported but needs to be tested.
#### Choose the encoder algorithm.
There are four options:
- baseline: it used a smaller transformer model to replace the bert model and with transformer summarization layer
- classifier: it uses pretrained BERT and fine-tune BERT with **simple logistic classification** summarization layer
- transformer: it uses pretrained BERT and fine-tune BERT with **transformer** summarization layer
- RNN: it uses pretrained BERT and fine-tune BERT with **LSTM** summarization layer

In [6]:
# notebook parameters
DATA_FOLDER = "./temp"
CACHE_DIR = "./temp"
DEVICE = "cuda"
BATCH_SIZE = 3000
NUM_GPUS = 4
encoder = "transformer"
model_name = "distilbert-base-uncased"

In [12]:
summarizer = ExtractiveSummarizer(model_name, encoder, CACHE_DIR)

I1210 21:49:41.613120 140261726336832 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at ./temp/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.1ccd1a11c9ff276830e114ea477ea2407100f4a3be7bdc45d37be9e37fa71c7e
I1210 21:49:41.615161 140261726336832 configuration_utils.py:168] Model config {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": null,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "n_heads": 12,
  "n_layers": 6,
  "num_labels": 0,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torchscript": false,
  "use_bfloat16": false,
  "vocab_size": 30522
}

I1210 21:49:41.755493 140261726336832 modeling_u

In [8]:
# batch_size is the number of tokens in a batch
train_dataloader = get_dataloader(train_iter(), is_labeled=True, batch_size=3000)

In [9]:
device, num_gpus = get_device(num_gpus=4, local_rank=-1)

In [37]:
epoch_iterator = tqdm(train_data_iterator)
i = 0
for step, batch in enumerate(epoch_iterator):
    batch = batch.to(device)
    i += 1
    print(i)
    if i > 10:
        break


NameError: name 'tqdm' is not defined

In [9]:
summarizer.fit(
            train_dataloader,
            num_gpus=4,
            gradient_accumulation_steps=2,
            max_steps=5e4,
            lr=2e-3,
            warmup_steps=1e4*0.5,
            verbose=True,
            report_every=200,
            clip_grad_norm=False,
        )

DataParallel


/dadendev/anaconda3/envs/cm3/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


loss: 5.858079, time: 47.937159, number of examples: 5, step 100 out of total 50000
loss: 5.761587, time: 36.250811, number of examples: 5, step 200 out of total 50000
loss: 5.445527, time: 36.161208, number of examples: 5, step 300 out of total 50000


KeyboardInterrupt: 

In [ ]:
DEVICE = "cuda"
summarizer.fit2(
            train_iter,
            device= DEVICE,
            batch_size=3000,
            num_gpus=NUM_GPUS,
            gradient_accumulation_steps=2,
            max_steps=5e4,
            lr=2e-3,
            warmup_steps=1e4*0.5,
            verbose=True,
            report_every=100,
        )

cuda
loss: 41.687942, time: 10.571247, number of examples: 5, step 100 out of total 50000
loss: 27.198075, time: 10.414026, number of examples: 5, step 200 out of total 50000
loss: 19.797057, time: 10.415758, number of examples: 5, step 300 out of total 50000
loss: 18.425389, time: 10.781584, number of examples: 6, step 400 out of total 50000
loss: 18.224794, time: 10.541868, number of examples: 5, step 500 out of total 50000
loss: 17.613259, time: 10.589073, number of examples: 5, step 600 out of total 50000
loss: 17.301366, time: 10.407273, number of examples: 5, step 700 out of total 50000
loss: 17.202934, time: 10.942506, number of examples: 5, step 800 out of total 50000
loss: 16.892723, time: 10.530933, number of examples: 5, step 900 out of total 50000
loss: 16.507498, time: 10.463890, number of examples: 5, step 1000 out of total 50000
loss: 16.465361, time: 10.396509, number of examples: 5, step 1100 out of total 50000
loss: 16.058329, time: 11.084911, number of examples: 5, s

In [14]:
import torch
torch.save(summarizer.model, model_name+'_newcommond'+"_dataparallel"+"_bertsum_processed_data.pt")

In [13]:
import torch
summarizer.model = torch.load("cnndm_transformersum_distilbert-base-uncased_bertsum_processed_data.pt")

### Model Evaluation

[ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)), or Recall-Oriented Understudy for Gisting Evaluation has been commonly used for evaluation text summarization.

In [14]:
test_dataset=[]
for i in test_iter():
    test_dataset.extend(i)

In [15]:
len(test_dataset)

11489

In [16]:
prediction = summarizer.predict(get_data_iter(test_dataset), device=DEVICE)

In [17]:
target = [test_dataset[i]['tgt_txt'] for i in range(len(test_dataset))]

In [18]:
rouge_transformer = get_rouge(prediction, target, "./results/")

11489
11489


2019-12-10 21:54:15,589 [MainThread  ] [INFO ]  Writing summaries.
I1210 21:54:15.589290 140261726336832 pyrouge.py:525] Writing summaries.
2019-12-10 21:54:15,591 [MainThread  ] [INFO ]  Processing summaries. Saving system files to ./results/tmpfmn7fdbj/system and model files to ./results/tmpfmn7fdbj/model.
I1210 21:54:15.591191 140261726336832 pyrouge.py:518] Processing summaries. Saving system files to ./results/tmpfmn7fdbj/system and model files to ./results/tmpfmn7fdbj/model.
2019-12-10 21:54:15,594 [MainThread  ] [INFO ]  Processing files in ./results/rouge-tmp-2019-12-10-21-54-14/candidate/.
I1210 21:54:15.594782 140261726336832 pyrouge.py:43] Processing files in ./results/rouge-tmp-2019-12-10-21-54-14/candidate/.
2019-12-10 21:54:16,668 [MainThread  ] [INFO ]  Saved processed files to ./results/tmpfmn7fdbj/system.
I1210 21:54:16.668018 140261726336832 pyrouge.py:53] Saved processed files to ./results/tmpfmn7fdbj/system.
2019-12-10 21:54:16,669 [MainThread  ] [INFO ]  Processing

---------------------------------------------
1 ROUGE-1 Average_R: 0.54208 (95%-conf.int. 0.53930 - 0.54484)
1 ROUGE-1 Average_P: 0.36866 (95%-conf.int. 0.36651 - 0.37103)
1 ROUGE-1 Average_F: 0.42466 (95%-conf.int. 0.42276 - 0.42672)
---------------------------------------------
1 ROUGE-2 Average_R: 0.24754 (95%-conf.int. 0.24499 - 0.25011)
1 ROUGE-2 Average_P: 0.16856 (95%-conf.int. 0.16669 - 0.17049)
1 ROUGE-2 Average_F: 0.19382 (95%-conf.int. 0.19190 - 0.19576)
---------------------------------------------
1 ROUGE-L Average_R: 0.49419 (95%-conf.int. 0.49159 - 0.49685)
1 ROUGE-L Average_P: 0.33667 (95%-conf.int. 0.33456 - 0.33889)
1 ROUGE-L Average_F: 0.38754 (95%-conf.int. 0.38561 - 0.38960)



In [30]:

rouge_transformer = get_rouge(prediction, target, "./results/")

11489
11489


2019-12-07 13:34:57,093 [MainThread  ] [INFO ]  Writing summaries.
I1207 13:34:57.093963 139683323610944 pyrouge.py:525] Writing summaries.
2019-12-07 13:34:57,095 [MainThread  ] [INFO ]  Processing summaries. Saving system files to ./results/tmpcferef5_/system and model files to ./results/tmpcferef5_/model.
I1207 13:34:57.095779 139683323610944 pyrouge.py:518] Processing summaries. Saving system files to ./results/tmpcferef5_/system and model files to ./results/tmpcferef5_/model.
2019-12-07 13:34:57,096 [MainThread  ] [INFO ]  Processing files in ./results/rouge-tmp-2019-12-07-13-34-56/candidate/.
I1207 13:34:57.096807 139683323610944 pyrouge.py:43] Processing files in ./results/rouge-tmp-2019-12-07-13-34-56/candidate/.
2019-12-07 13:34:58,251 [MainThread  ] [INFO ]  Saved processed files to ./results/tmpcferef5_/system.
I1207 13:34:58.251595 139683323610944 pyrouge.py:53] Saved processed files to ./results/tmpcferef5_/system.
2019-12-07 13:34:58,253 [MainThread  ] [INFO ]  Processing

---------------------------------------------
1 ROUGE-1 Average_R: 0.53608 (95%-conf.int. 0.53328 - 0.53883)
1 ROUGE-1 Average_P: 0.37832 (95%-conf.int. 0.37594 - 0.38073)
1 ROUGE-1 Average_F: 0.42902 (95%-conf.int. 0.42696 - 0.43104)
---------------------------------------------
1 ROUGE-2 Average_R: 0.24854 (95%-conf.int. 0.24592 - 0.25123)
1 ROUGE-2 Average_P: 0.17574 (95%-conf.int. 0.17360 - 0.17792)
1 ROUGE-2 Average_F: 0.19879 (95%-conf.int. 0.19676 - 0.20102)
---------------------------------------------
1 ROUGE-L Average_R: 0.48980 (95%-conf.int. 0.48709 - 0.49253)
1 ROUGE-L Average_P: 0.34635 (95%-conf.int. 0.34386 - 0.34873)
1 ROUGE-L Average_F: 0.39242 (95%-conf.int. 0.39039 - 0.39448)



In [32]:
%history

## Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs.
QUICK_RUN = True
USE_PREPROCESSED_DATA = True
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
%load_ext autoreload
%autoreload 2
import sys
import os

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)
sys.path.insert(0, "./")
sys.path.insert(0, "/dadendev/nlp/examples/text_summarization/BertSum/")
if USE_PREPROCESSED_DATA:
    data_path = "./temp_data5/"
    if not os.path.exists(data_path):
        os.mkdir(data_path)
    #CNNDMBertSumProcessedData.download(local_path=data_path)
    train_iter, test_iter = CNNDMBertSumProcessedData().splits(root=data_path)
data_path = "./temp_data4/"
from utils_nlp.dataset.cnndm import CNNDMBertSumProcessedData

train_files = CNNDMBertSumProcessedData.save_data(
    ext_sum_train, is_test=False, path_and_prefix=data_path, chunk_s

In [53]:
#from utils_nlp.eval.evaluate_summarization import get_rouge
#rouge_transformer = get_rouge(prediction, target, "./results/")

2019-10-31 17:59:07,705 [MainThread  ] [INFO ]  Writing summaries.
I1031 17:59:07.705975 140308661311296 pyrouge.py:525] Writing summaries.
2019-10-31 17:59:07,707 [MainThread  ] [INFO ]  Processing summaries. Saving system files to ./results/tmp2jh52kci/system and model files to ./results/tmp2jh52kci/model.
I1031 17:59:07.707562 140308661311296 pyrouge.py:518] Processing summaries. Saving system files to ./results/tmp2jh52kci/system and model files to ./results/tmp2jh52kci/model.
2019-10-31 17:59:07,708 [MainThread  ] [INFO ]  Processing files in ./results/rouge-tmp-2019-10-31-17-59-07/candidate/.
I1031 17:59:07.708495 140308661311296 pyrouge.py:43] Processing files in ./results/rouge-tmp-2019-10-31-17-59-07/candidate/.
2019-10-31 17:59:07,719 [MainThread  ] [INFO ]  Saved processed files to ./results/tmp2jh52kci/system.
I1031 17:59:07.719068 140308661311296 pyrouge.py:53] Saved processed files to ./results/tmp2jh52kci/system.
2019-10-31 17:59:07,720 [MainThread  ] [INFO ]  Processing

100
100
---------------------------------------------
1 ROUGE-1 Average_R: 0.41601 (95%-conf.int. 0.38468 - 0.44964)
1 ROUGE-1 Average_P: 0.21124 (95%-conf.int. 0.19304 - 0.22896)
1 ROUGE-1 Average_F: 0.27238 (95%-conf.int. 0.25137 - 0.29249)
---------------------------------------------
1 ROUGE-2 Average_R: 0.13888 (95%-conf.int. 0.11130 - 0.16833)
1 ROUGE-2 Average_P: 0.06738 (95%-conf.int. 0.05454 - 0.08099)
1 ROUGE-2 Average_F: 0.08825 (95%-conf.int. 0.07190 - 0.10610)
---------------------------------------------
1 ROUGE-L Average_R: 0.37098 (95%-conf.int. 0.34192 - 0.40359)
1 ROUGE-L Average_P: 0.18856 (95%-conf.int. 0.17234 - 0.20503)
1 ROUGE-L Average_F: 0.24308 (95%-conf.int. 0.22405 - 0.26304)



In [57]:
test_dataset[0]['tgt_txt']

'andrew mogni , 20 , from glen ellyn , illinois , had only just arrived for a semester program when the incident happened in january<q>he was flown back to chicago via air on march 20 but he died on sunday<q>initial police reports indicated the fall was an accident but authorities are investigating the possibility that mogni was robbed<q>his cousin claims he was attacked and thrown 40ft from a bridge'

In [56]:
prediction[0]

'andrew mogni , 20 , from glen ellyn , illinois , had only just arrived for a semester program in italy when the incident happened in january .<q>he was flown back to chicago via air ambulance on march 20 , but he died on sunday .<q>a university of iowa student has died nearly three months after a fall in rome in a suspected robbery attack in rome .'

In [26]:
test_dataset[0]['src_txt']

['a university of iowa student has died nearly three months after a fall in rome in a suspected robbery attack in rome .',
 'andrew mogni , 20 , from glen ellyn , illinois , had only just arrived for a semester program in italy when the incident happened in january .',
 'he was flown back to chicago via air ambulance on march 20 , but he died on sunday .',
 'andrew mogni , 20 , from glen ellyn , illinois , a university of iowa student has died nearly three months after a fall in rome in a suspected robbery',
 'he was taken to a medical facility in the chicago area , close to his family home in glen ellyn .',
 "he died on sunday at northwestern memorial hospital - medical examiner 's office spokesman frank shuftan says a cause of death wo n't be released until monday at the earliest .",
 'initial police reports indicated the fall was an accident but authorities are investigating the possibility that mogni was robbed .',
 "on sunday , his cousin abby wrote online : ` this morning my cous